In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import requests
import pandas as pd
import math
import folium
from fuzzywuzzy import process

In [2]:
from shapely.geometry import Point
import geopandas as gpd

## Datos de BiciMAD

In [36]:
engine = create_engine('mysql://ironhack_user:%Vq=c>G5@173.201.189.217/BiciMAD')
inspector = inspect(engine)
inspector.get_table_names()

['bicimad_stations', 'bicipark_stations']

In [37]:
bicimad = pd.read_sql_query("SELECT * FROM bicimad_stations", engine)
bicimad

,id,name,light,number,address,activate,no_available,total_bases,dock_bikes,free_bases,reservations_count,geometry.type,geometry.coordinates
0,1,1a - Puerta del Sol A,3,1a,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.7018341, 40.4172137]"
1,2,1b - Puerta del Sol B,3,1b,Puerta del Sol nº 1,1,1,30,0,0,0,Point,"[-3.701602938060457, 40.41731271011562]"
2,3,2 - Miguel Moya,3,2,Calle Miguel Moya nº 1,1,1,24,0,0,0,Point,"[-3.7058415, 40.4205886]"
3,4,3 - Plaza Conde Suchil,2,3,Plaza del Conde del Valle de Súchil nº 3,1,0,18,9,9,0,Point,"[-3.7069171, 40.4302937]"
4,5,4 - Malasaña,1,4,Calle Manuela Malasaña nº 5,1,0,24,23,1,0,Point,"[-3.7025875, 40.4285524]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,265,257 - INEF,0,257,Avenida Juan de Herrera frente a la calle Paul...,1,0,24,3,19,0,Point,"[-3.72997, 40.43896]"
260,266,258 - Ciudad Universitaria 1,2,258,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,9,14,0,Point,"[-3.72699, 40.44375]"
261,267,259 - Ciudad Universitaria 2,2,259,Avenida de la Complutense (Metro Ciudad Univer...,1,0,24,12,10,0,Point,"[-3.72693, 40.44342]"
262,268,260 - Facultad Biología,2,260,Calle José Antonio Novais frente al nº 12,1,0,24,12,10,0,Point,"[-3.7272945, 40.4483322]"


## Datos de Centros de Atención Médica

In [5]:
response = requests.get('https://datos.madrid.es/egob/catalogo/212769-0-atencion-medica.json')
json = response.json()
json.keys()

dict_keys(['@context', '@graph'])

In [6]:
json['@graph']

[{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/53123-centro-concertado-atencion-adicciones-ccad-casa-campo-cruz-roja-.json',
  '@type': 'https://datos.madrid.es/egob/kos/entidadesYorganismos/CentrosAtencionMedica/CentrosAsistenciaDrogodependientes',
  'id': '53123',
  'title': 'Centro Concertado de Atención a las Adicciones (CCAD) Casa de Campo (Cruz Roja)',
  'relation': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=bfa48ab43d6bb410VgnVCM100000171f5a0aRCRD&vgnextoid=97d3598d1351c010VgnVCM1000000b205a0aRCRD',
  'address': {'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Latina'},
   'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Latina/Barrio/PuertaAngel'},
   'locality': 'MADRID',
   'postal-code': '28011',
   'street-address': 'AVENIDA PORTUGAL'},
  'location': {'latitude': 40.41633059885888,
   'longitude': -3.7239914257003304},
  'organization': {'organi

In [7]:
ayto = pd.DataFrame(pd.json_normalize(json['@graph']))
ayto

,@id,@type,id,title,relation,address.district.@id,address.area.@id,address.locality,address.postal-code,address.street-address,location.latitude,location.longitude,organization.organization-desc,organization.accesibility,organization.schedule,organization.services,organization.organization-name
0,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,53123,Centro Concertado de Atención a las Adicciones...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28011,AVENIDA PORTUGAL,40.416331,-3.723991,El CCAD Casa de Campo que forma parte de la r...,1,De lunes a viernes de 9 a 17 horas,Valoración interdisciplinar del paciente. Trat...,Centro Concertado de Atención a las Adicciones...
1,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,134929,Centro Concertado de Atención a las Adicciones...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28002,CALLE SANTA HORTENSIA 1,40.446053,-3.670277,El CCAD Cáritas que forma parte de la red del ...,1,"De lunes a viernes mañanas de 9 a 14 horas, ta...",Valoración interdisciplinar del/de la paciente...,Centro Concertado de Atención a las Adicciones...
2,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11867,Centro de Atención a las Adicciones (CAD) de A...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28045,PASEO CHOPERA 12,40.393057,-3.698067,"Forma de acceso: directo, a través de los ser...",1,Horario ordinario: De lunes a jueves de 8:30 ...,Valoración por equipo interdisciplinar Tratami...,Centro de Atención a las Adicciones (CAD) de A...
3,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11887,Centro de Atención a las Adicciones (CAD) de H...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28043,CALLE MINAYA 7,40.468289,-3.636150,"Acceso: directo, a través de los servicios mu...",1,Horario ordinario: De lunes a jueves de 8:30 ...,Valoración por equipo interdisciplinar Tratami...,Centro de Atención a las Adicciones (CAD) de H...
4,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,11862,Centro de Atención a las Adicciones (CAD) de L...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28047,CALLE CAMARENA 10,40.395971,-3.756272,"Acceso: directo, a traves de los servicios mu...",1,Horario ordinario: De lunes a jueves de 8:30 ...,Valoración por equipo interdisciplinar Tratami...,Centro de Atención a las Adicciones (CAD) de L...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,8835,Servicio de Inmunoprofilaxis de la Rabia Humana,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Provincia/Mad...,MADRID,28006,CALLE MONTESA 22,40.428977,-3.672161,"Metro: Manuel BecerraBus: 1 , 2 , 12 , 21 , ...",1,De lunes a viernes de 9 a 14 horas,Vacunación antirrábica preexposición (viajeros...,Servicio de Inmunoprofilaxis de la Rabia Humana
267,https://datos.madrid.es/egob/catalogo/tipo/ent...,https://datos.madrid.es/egob/kos/entidadesYorg...,7318822,Unidad de Salud Mental de niños y adolescentes...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,https://datos.madrid.es/egob/kos/Provincia/Mad...,https://datos.madrid.es/egob/kos/Pr

## Creación de tabla final

In [8]:
ayto[['title', 'address.street-address']]

,title,address.street-address
0,Centro Concertado de Atención a las Adicciones...,AVENIDA PORTUGAL
1,Centro Concertado de Atención a las Adicciones...,CALLE SANTA HORTENSIA 1
2,Centro de Atención a las Adicciones (CAD) de A...,PASEO CHOPERA 12
3,Centro de Atención a las Adicciones (CAD) de H...,CALLE MINAYA 7
4,Centro de Atención a las Adicciones (CAD) de L...,CALLE CAMARENA 10
...,...,...
266,Servicio de Inmunoprofilaxis de la Rabia Humana,CALLE MONTESA 22
267,Unidad de Salud Mental de niños y adolescentes...,CALLE CANDIDO MATEOS 11
268,Unidad de Salud Mental de niños y adolescentes...,CALLE AGUILEÑAS 1
269,Unidad móvil Madroño,PASEO REY 36 Junto al albergue de San Isidro


In [9]:
main_table = pd.concat([ayto[['title', 'address.street-address']], bicimad[['name', 'address']]], axis=1)
main_table

,title,address.street-address,name,address
0,Centro Concertado de Atención a las Adicciones...,AVENIDA PORTUGAL,1a - Puerta del Sol A,Puerta del Sol nº 1
1,Centro Concertado de Atención a las Adicciones...,CALLE SANTA HORTENSIA 1,1b - Puerta del Sol B,Puerta del Sol nº 1
2,Centro de Atención a las Adicciones (CAD) de A...,PASEO CHOPERA 12,2 - Miguel Moya,Calle Miguel Moya nº 1
3,Centro de Atención a las Adicciones (CAD) de H...,CALLE MINAYA 7,3 - Plaza Conde Suchil,Plaza del Conde del Valle de Súchil nº 3
4,Centro de Atención a las Adicciones (CAD) de L...,CALLE CAMARENA 10,4 - Malasaña,Calle Manuela Malasaña nº 5
...,...,...,...,...
266,Servicio de Inmunoprofilaxis de la Rabia Humana,CALLE MONTESA 22,NaN,NaN
267,Unidad de Salud Mental de niños y adolescentes...,CALLE CANDIDO MATEOS 11,NaN,NaN
268,Unidad de Salud Mental de niños y adolescentes...,CALLE AGUILEÑAS 1,NaN,NaN
269,Unidad móvil Madroño,PASEO REY 36 Junto al albergue de San Isidro,NaN,NaN


## Funciones geocalculations

In [8]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [9]:
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

## Puntos

In [13]:
def start_points(ayto):
    
    start_points = []
    for centro in range(len(ayto)):

        start_point = (float(ayto['location.latitude'][centro]), float(ayto['location.longitude'][centro]))
        start_points.append(start_point)
    
    return start_points

def finish_points(bicimad):
    
    finish_points = []
    for parada in range(len(bicimad)):
        
        lat_long = bicimad['geometry.coordinates'][parada].split(', ')
        lat = lat_long[0][1:]
        long = lat_long[1][:-1]
        finish_point = (float(lat),float(long))
        finish_points.append(finish_point)

    return finish_points

start_points = start_points(ayto)
finish_points = finish_points(bicimad)


In [14]:
print(len(start_points))
print(len(finish_points))

271
264


## Función distancia mínima

In [15]:
def min_distance(start_point, finish_points):
    
    parada = 0
    min_distance = float(math.inf)
    index_distance = (-1,-1)
    for j in finish_points:
        
        distance = distance_meters(start_point[0], start_point[1],j[1],j[0])
        if distance.iloc[0] < min_distance:
            min_distance = distance.iloc[0]
            index_distance = (parada, min_distance)
        parada+=1
        
    return index_distance

In [16]:
distancias = [min_distance(i,finish_points) for i in start_points]

In [26]:
distancias

[(218, 560.8786937509844),
 (253, 435.7951954238147),
 (167, 58.729086969190405),
 (232, 3087.1089714142886),
 (226, 3423.901233993844),
 (217, 1991.3163714939578),
 (209, 990.0539428394632),
 (229, 138.23879611898576),
 (215, 4526.866973908478),
 (217, 2747.3642445890127),
 (157, 560.7166021677107),
 (215, 1166.521873992915),
 (223, 3758.9511551952273),
 (124, 210.30764059275256),
 (124, 210.30764059275256),
 (147, 140.94123534097136),
 (239, 417.725834316395),
 (115, 18.443395145519073),
 (226, 1060.869189796638),
 (214, 4562.908485005874),
 (232, 3114.634710334881),
 (223, 1541.1887239239961),
 (100, 81.15111255608684),
 (181, 107.46406419218553),
 (209, 79.43004393996438),
 (131, 289.0227791843529),
 (223, 103.72376548709335),
 (215, 2383.0221963904087),
 (217, 1696.079818712948),
 (263, 2701.7466640142457),
 (45, 188.74475050993476),
 (229, 557.6001254422033),
 (215, 4437.52762519913),
 (253, 1540.8521982284092),
 (147, 140.94123534097136),
 (263, 1246.146680538568),
 (147, 152.87

## Creación de la tabla

In [34]:
ayto[['title','address.street-address']]
ayto['parada'] = ''
ayto['dirección parada'] = ''
ayto ['distancia (m)'] = ''

for i in range(len(ayto)):
    
    ayto['parada'].loc[i] = bicimad['name'].loc[distancias[i][0]]
    ayto['dirección parada'].loc[i] = bicimad['address'].loc[distancias[i][0]]
    ayto ['distancia (m)'].loc[i] = distancias[i][1]

main_table = ayto[['title','address.street-address','parada','dirección parada', 'distancia (m)']]
main_table = main_table.rename({'title': 'Place of Interest', 'address.street-address':'Place Address', 'parada':'BiciMAD Station', 'dirección parada':'Station Location', 'distancia (m)':'distance (m)'}, axis=1)
main_table.to_csv('../data/main_table.csv')
main_table

C:\Users\pedro\AppData\Local\Temp\ipykernel_23580\237595645.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ayto['parada'].loc[i] = bicimad['name'].loc[distancias[i][0]]
C:\Users\pedro\AppData\Local\Temp\ipykernel_23580\237595645.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ayto['dirección parada'].loc[i] = bicimad['address'].loc[distancias[i][0]]
C:\Users\pedro\AppData\Local\Temp\ipykernel_23580\237595645.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

,Place of Interest,Place Address,BiciMAD Station,Station Location,distance (m)
0,Centro Concertado de Atención a las Adicciones...,AVENIDA PORTUGAL,216 - Puerta del Ángel,Plaza Puerta del Ángel nº 4,560.878694
1,Centro Concertado de Atención a las Adicciones...,CALLE SANTA HORTENSIA 1,251 - Corazón de María,Calle Santa Hortensia nº 31,435.795195
2,Centro de Atención a las Adicciones (CAD) de A...,PASEO CHOPERA 12,165 - Entrada Matadero,Paseo de la Chopera nº 14,58.729087
3,Centro de Atención a las Adicciones (CAD) de H...,CALLE MINAYA 7,230 - Carlota O’Neill,Calle Carlota O'Neill nº 32,3087.108971
4,Centro de Atención a las Adicciones (CAD) de L...,CALLE CAMARENA 10,224 - Caramuel,Calle Caramuel nº 51,3423.901234
...,...,...,...,...,...
266,Servicio de Inmunoprofilaxis de la Rabia Humana,CALLE MONTESA 22,145 - Ortega y Gasset 87,Calle Ortega y Gasset nº 87,140.941235
267,Unidad de Salud Mental de niños y adolescentes...,CALLE CANDIDO MATEOS 11,239 - Francos Rodríguez,Calle Francos Rodríguez nº 62,2112.951861
268,Unidad de Salud Mental de niños y adolescentes...,CALLE AGUILEÑAS 1,207 - Remonta,Calle Azucenas nº 1,494.4184
269,Unidad móvil Madroño,PASEO REY 36 Junto al albergue de San Isidro,161 - Paseo de la Florida,Paseo de la Florida nº 8,529.669171


## Pruebas

In [11]:

print(bicimad['geometry.coordinates'][2])
lat = bicimad['geometry.coordinates'][2].split(', ')
lat = lat[0][1:]
lat

long = bicimad['geometry.coordinates'][2].split(', ')
long = long[1][:-1]
long

[-3.7058415, 40.4205886]


'40.4205886'

In [64]:
prueba = to_mercator(23.444, 33.333)
print(prueba)

0    POINT (2609774.142 3939588.017)
dtype: geometry


In [13]:
len(datos)

528

In [49]:
dataframe=pd.DataFrame({'Attendance': {0: 60, 1: 100, 2: 80,3: 78,4: 95},
                        'Name': {0: 'Olivia', 1: 'John', 2: 'Laura',3: 'Ben',4: 'Kevin'},
                        'Obtained Marks': {0: 90, 1: 75, 2: 82, 3: 64, 4: 45}})
dataframe[dataframe['Name'] == 'Olivia']

,Attendance,Name,Obtained Marks
0,60,Olivia,90


In [51]:
dataframe.to_csv('./data/main_table.csv')

OSError: Cannot save file into a non-existent directory: 'data'

In [19]:
dataframe = pd.read_csv('../data/main_table.csv')
dataframe.iloc()

In [28]:
dataframe.head(10)

,Unnamed: 0,Place of Interest,Place Address,BiciMAD Station,Station Location,distance (m)
0,0,Centro Concertado de Atención a las Adicciones...,AVENIDA PORTUGAL,216 - Puerta del Ángel,Plaza Puerta del Ángel nº 4,560.878694
1,1,Centro Concertado de Atención a las Adicciones...,CALLE SANTA HORTENSIA 1,251 - Corazón de María,Calle Santa Hortensia nº 31,435.795195
2,2,Centro de Atención a las Adicciones (CAD) de A...,PASEO CHOPERA 12,165 - Entrada Matadero,Paseo de la Chopera nº 14,58.729087
3,3,Centro de Atención a las Adicciones (CAD) de H...,CALLE MINAYA 7,230 - Carlota O’Neill,Calle Carlota O'Neill nº 32,3087.108971
4,4,Centro de Atención a las Adicciones (CAD) de L...,CALLE CAMARENA 10,224 - Caramuel,Calle Caramuel nº 51,3423.901234
5,5,Centro de Atención a las Adicciones (CAD) de S...,CALLE ALCALA 527,215 - Gutierre de Cetina,Calle Gutierre de Cetina nº 77,1991.316371
6,6,Centro de Atención a las Adicciones (CAD) de T...,CALLE PINOS ALTA 122,207 - Remonta,Calle Azucenas nº 1,990.053943
7,7,Centro de Atención a las Adicciones (CAD) de V...,CALLE CONCORDIA 17,227 - Concordia,Calle Arroyo del Olivar nº 6,138.238796
8,8,Centro de Atención a las Adicciones (CAD) de V...,CALLE VERDE VIENTO 16,213 - Glorieta de Cádiz,Calle Antonio López nº 129,4526.866974
9,9,Centro de Atención Integral a Drogodependiente...,PLAZA CIVICA esquina C/ Pobladura del Valle,215 - Gutierre de Cetina,Calle Gutierre de Cetina nº 77,2747.364245


In [27]:
dataframe[['Place of Interest','BiciMAD Station']]

,Place of Interest,BiciMAD Station
0,Centro Concertado de Atención a las Adicciones...,216 - Puerta del Ángel
1,Centro Concertado de Atención a las Adicciones...,251 - Corazón de María
2,Centro de Atención a las Adicciones (CAD) de A...,165 - Entrada Matadero
3,Centro de Atención a las Adicciones (CAD) de H...,230 - Carlota O’Neill
4,Centro de Atención a las Adicciones (CAD) de L...,224 - Caramuel
...,...,...
266,Servicio de Inmunoprofilaxis de la Rabia Humana,145 - Ortega y Gasset 87
267,Unidad de Salud Mental de niños y adolescentes...,239 - Francos Rodríguez
268,Unidad de Salud Mental de niños y adolescentes...,207 - Remonta
269,Unidad móvil Madroño,161 - Paseo de la Florida


In [63]:
nombre = input('nombre:')
nombre_centros = [dataframe['Place of Interest'][i] for i in range(len(dataframe))]
aprox = process.extractOne(nombre, nombre_centros)
print(aprox)
nombre = aprox[0]
print(nombre)
dataframe1 = dataframe[dataframe['Place of Interest'] == nombre]
if aprox[1] < 90:
    print('No se ha encontrado ninguna coincidencia')
else:
    print(dataframe1)


nombre:a
('Centro Concertado de Atención a las Adicciones (CCAD) Casa de Campo (Cruz Roja)', 60)
Centro Concertado de Atención a las Adicciones (CCAD) Casa de Campo (Cruz Roja)
No se ha encontrado ninguna coincidencia


In [4]:
m = folium.Map(location=[40.416948896201355, -3.7038645330446434], zoom_start=12)
m

In [5]:
folium.Marker(
    location=[40.38689777918933, -3.6900368792997886],
    popup="legazpi",
    icon=folium.Icon(icon="bicycle", prefix='fa', color='red'),
).add_to(m)
m